### Generate VW format input files
For train,test,dev files

In [4]:
# Get all the tags in hash list
hash = {}
tag_file_name = '../Dep_ws/ptb_tags'
# input_file_name = './sanskrit_treebank/ud_pos_ner_dp_train_san'
files = ['../Dep_ws/ptb3.0-stanford.gold.cpos.test.conll']

def readtags():
    for line in open(tag_file_name).readlines():
        hash[line.split()[0]] = int(line.strip().split()[1])
tag_set = set()
c = 1
readtags()
for input_file_name in files:
    data = open(input_file_name).readlines()
    writer = open(input_file_name+'.vw','w')
    for line in data:
        if line == '\n':
            writer.write('\n')
            continue
        splits = line.strip().lower().split()
        strw = "|w %s"%splits[1].replace(":","COL");
        strp = "|p %s"%splits[3].replace(":","COL");
        tag = splits[7]
        tag_set.add(tag)
        if tag not in hash:
            hash[tag] = c
            print(tag)
            c+=1
        writer.write('%s %s %s:%s%s %s\n' % (int(splits[6]), hash[tag], int(splits[6]), tag, strw, strp))
    writer.close()
print('Value of out of voc tag ',c)
print('successful')

('Value of out of voc tag ', 1)
successful


In [12]:
# j=1
# for t in tag_set:
#     print t, j
#     j=j+1

### Create dict of MA because space is creating problem
So change all tags to integer mapping

In [17]:
i=0
tag_dict = dict()
for tag in tag_set:
    tag_dict[tag] = i
    i = i+1

### Modify evaluate script:

In [5]:
#!/usr/bin/env python
# evaluation script modified from redshift parser

import os
import sys
from collections import defaultdict

def pc(num, den):
    return (num / float(den+1e-100)) * 100

def fmt_acc(label, n, l_corr, u_corr, total_errs):
    l_pc = pc(l_corr, n)
    u_pc = pc(u_corr, n)
    err_pc = pc(n - l_corr, total_errs)
    return '%s\t%d\t%.3f\t%.3f\t%.3f' % (label, n, l_pc, u_pc, err_pc)


def gen_toks(loc):
    sent_strs = open(str(loc)).read().strip().split('\n\n')
    token = None
    i = 0
    for sent_str in sent_strs:
        # print(sent_str)
        tokens = [Token(i, tok_str.split()) for i, tok_str in enumerate(sent_str.split('\n'))]
        for token in tokens:
            yield sent_str, token

def gen_ptb_toks(loc):
    sent_strs = open(str(loc)).read().strip().split('\n\n')
    token = None
    i = 0
    for sent_str in sent_strs:
        # print(sent_str)
        tokens = [ptb_Token(i, tok_str.split()) for i, tok_str in enumerate(sent_str.split('\n'))]
        for token in tokens:
            yield sent_str, token

class ptb_Token(object):
    def __init__(self, id_, attrs):
        self.id = id_
        self.label = attrs[-3]
        if self.label.lower() == 'root':
            self.label = 'ROOT'
        try:
            head = int(attrs[-4])
        except:
            try:
                self.label = 'P'
                head = int(attrs[-4])
            except:
                print attrs
                raise
        self.head = head
        self.pos = attrs[3]
        self.word = attrs[1]
        self.dir = 'R' if head >= 0 and head < self.id else 'L'
        
class Token(object):
    def __init__(self, id_, attrs):
        self.id = id_
        # CoNLL format
        if len(attrs) == 10:
            new_attrs = [str(int(attrs[0]) - 1)]
            new_attrs.append(attrs[1])
            new_attrs.append(attrs[3])
            new_attrs.append(str(int(attrs[-4]) - 1))
            new_attrs.append(attrs[-3])
            attrs = new_attrs
        self.label = attrs[-1]
        if self.label.lower() == 'root':
            self.label = 'ROOT'
        try:
            head = int(attrs[-2])
        except:
            try:
                self.label = 'P'
                head = int(attrs[-1])
            except:
                print attrs
                raise
        attrs.pop()
        attrs.pop()
        self.head = head
        self.pos = attrs.pop()
        self.word = attrs.pop()
        self.dir = 'R' if head >= 0 and head < self.id else 'L'

test_loc = '../Dep_ws/dep.test.parse'
gold_loc = '../Dep_ws/ptb3.0-stanford.gold.cpos.test.conll'
n_by_label = defaultdict(lambda: defaultdict(int))
u_by_label = defaultdict(lambda: defaultdict(int))
l_by_label = defaultdict(lambda: defaultdict(int))
N = 0
u_nc = 0
l_nc = 0
for (sst, t), (ss, g) in zip(gen_toks(test_loc), gen_ptb_toks(gold_loc)):
#     not eval_punct and 
    if g.word in ",.-;:'\"!?`{}()[]":
        continue
    prev_g = g
    prev_t = t
    u_c = g.head == t.head
    l_c = u_c and g.label.lower() == t.label.lower()
    N += 1
    l_nc += l_c
    u_nc += u_c
    n_by_label[g.dir][g.label] += 1
    u_by_label[g.dir][g.label] += u_c
    l_by_label[g.dir][g.label] += l_c
n_l_err = N - l_nc
for D in ['L', 'R']:
    n_other = 0
    l_other = 0
    u_other = 0
    for label, n in sorted(n_by_label[D].items(), key=lambda i: i[1], reverse=True):
        if n == 0:
            continue
        elif n < 100:
            n_other += n
            l_other += l_by_label[D][label]
            u_other += u_by_label[D][label]
        else:
            l_corr = l_by_label[D][label]
            u_corr = u_by_label[D][label]
print 'U: %.3f' % pc(u_nc, N)
print 'L: %.3f' % pc(l_nc, N)



U: 86.720
L: 84.313
